## Step 6: 수동 역전파

앞서 이론적으로 이해했던 역전파를 Variable과 Function 클래스를 확장하여 직접 구현해보도록 하겠다.

### 6.1 Variable 클래스 추가 구현

역전파에 대응하는 Variable 클래스를 구현하기 위해 grad 인스턴스도 추가하도록 하겠다.

```python
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
```

grad에는 None을 할당해 초기화를 해두었다. 이후 계산 과정에서 실제로 역전파가 진행되게 되면 미분값을 계산해서 여기에 대입하게 된다.

### 6.2 Function 클래스 추가 구현

이전 단계까지의 Function 클래스는 일반적인 계산인 순전파(forward)만을 지원했다. 여기에 다음 두 가지 기능을 추가하도록 하겠다.
- 미분을 계산하는 역전파(backward) 메서드
- forward 메서드 호출 시 건네받은 Variable 인스턴스 유지

```python
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input # 입력 변수를 기억
        return output
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()
```

위 `__call__` 메서드에서 input을 인스턴스 변수 `self.input`에 저장한 이유는 추후 `backward` 메서드에서 사용하기 위함이다.

### 6.3 Square과 Exp 클래스 추가 구현

Function을 상속한 함수에서 역전파를 구현해보도록 하자.

Square 클래스는 $y=x^2$을 계산하는 함수이고, 이를 미분하면 ${dy\over dx}=2x$가 된다.

```python
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx
```

역전파를 담당하는 backward 메서드의 인수 gy는 ndarray 인스턴스이며, 출력 쪽에서 전해지는 미분값을 전달하는 역할을 한다. 이렇게 전달된 미분에 '$y=x^2$의 미분'을 곱한 값이 backward의 결과가 된다.

이어서 $y = e^x$에 대해서도 역전파 구현을 진행해보도록 하자. 이 계산의 미분은 ${dy \over dx} = e^x$이다.

```python
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx
```

### 6.4 역전파 구현

다음 순전파 계산을 역전파 해보도록 하겠다.

$$
{e^{x^2}}^2
$$

먼저 순전파를 진행하는 코드는 다음과 같다.

```python
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
```

이어서 역전파로 미분해보도록 하겠다. 순전파 때와는 반대 방향으로 각 함수의 backward 메서드를 호출하면 된다.

```python
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)

print(x.grad)
```
\> 3.297442541400256

역전파는 ${dy\over dx} = 1$에서 시작한다. 따라서 출력 y의 미분 값을 np.array(1.0)으로 설정한다. 이후 C → B → A 순으로 backward 메서드를 호출하면 각 변수에 대한 미분값이 구해진다.

이와 같이 역전파를 구현할 수 있다. 하지만 이를 수동으로 일일이 작성하는 것은 여간 불편한 것이 아니다. 따라서 다음 스텝에서는 이를 자동화해보도록 하겠다.